In [5]:
import pandas as pd
import geopandas as gpd
from shapely import geometry
import mercantile
from tqdm import tqdm
import os
import tempfile

## Step 1 - Define our area of interest (AOI)

We define our area of interest (or AOI) as a GeoJSON geometry, then use the `shapely` library to get the bounding box.

**Note**: the coordinate reference system for the GeoJSON should be "EPSG:4326", i.e. in global lat/lon format.

In [7]:
# Geometry copied from https://geojson.io
aoi_geom = {
    "coordinates": [
        [
            [
              28.28212184503178,
              -26.116476754935455
            ],
            [
              28.28189785219746,
              -26.116756799833695
            ],
            [
              28.282306142934033,
              -26.11716922837192
            ],
            [
              28.28283918917262,
              -26.117217599524878
            ],
            [
              28.283559368665635,
              -26.116810262874203
            ],
            [
              28.28300931031248,
              -26.116789896005358
            ],
            [
              28.283029157779225,
              -26.11655313088339
            ],
            [
              28.28212184503178,
              -26.116476754935455
            ],
        ]
    ],
    "type": "Polygon",
}
aoi_shape = geometry.shape(aoi_geom)
minx, miny, maxx, maxy = aoi_shape.bounds

output_fn = "example_building_footprints.geojson"

In [ ]:
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              28.28212184503178,
              -26.116476754935455
            ],
            [
              28.28189785219746,
              -26.116756799833695
            ],
            [
              28.282306142934033,
              -26.11716922837192
            ],
            [
              28.28283918917262,
              -26.117217599524878
            ],
            [
              28.283559368665635,
              -26.116810262874203
            ],
            [
              28.28300931031248,
              -26.116789896005358
            ],
            [
              28.283029157779225,
              -26.11655313088339
            ],
            [
              28.28212184503178,
              -26.116476754935455
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

## Step 2 - Determine which tiles intersect our AOI

In [8]:
quad_keys = set()
for tile in list(mercantile.tiles(minx, miny, maxx, maxy, zooms=9)):
    quad_keys.add(mercantile.quadkey(tile))
quad_keys = list(quad_keys)
print(f"The input area spans {len(quad_keys)} tiles: {quad_keys}")

The input area spans 1 tiles: ['300301220']


## Step 3 - Download the building footprints for each tile that intersects our AOI and crop the results

This is where most of the magic happens. We download all the building footprints for each tile that intersects our AOI, then only keep the footprints that are _contained_ by our AOI.

*Note*: this step might take awhile depending on how many tiles your AOI covers and how many buildings footprints are in those tiles.

In [9]:
df = pd.read_csv(
    "https://minedbuildings.z5.web.core.windows.net/global-buildings/dataset-links.csv", dtype=str
)
df.head()

,Location,QuadKey,Url,Size,UploadDate
0,Abyei,122320113,https://minedbuildings.z5.web.core.windows.net...,74.5KB,2025-02-28
1,Abyei,122320131,https://minedbuildings.z5.web.core.windows.net...,8.3KB,2025-02-28
2,Abyei,122321002,https://minedbuildings.z5.web.core.windows.net...,392.2KB,2025-02-28
3,Abyei,122321003,https://minedbuildings.z5.web.core.windows.net...,72.8KB,2025-02-28
4,Abyei,122321020,https://minedbuildings.z5.web.core.windows.net...,1.2MB,2025-02-28


In [10]:
idx = 0
combined_gdf = gpd.GeoDataFrame()
with tempfile.TemporaryDirectory() as tmpdir:
    # Download the GeoJSON files for each tile that intersects the input geometry
    tmp_fns = []
    for quad_key in tqdm(quad_keys):
        rows = df[df["QuadKey"] == quad_key]
        if rows.shape[0] == 1:
            url = rows.iloc[0]["Url"]

            df2 = pd.read_json(url, lines=True)
            df2["geometry"] = df2["geometry"].apply(geometry.shape)

            gdf = gpd.GeoDataFrame(df2, crs=4326)
            fn = os.path.join(tmpdir, f"{quad_key}.geojson")
            tmp_fns.append(fn)
            if not os.path.exists(fn):
                gdf.to_file(fn, driver="GeoJSON")
        elif rows.shape[0] > 1:
            raise ValueError(f"Multiple rows found for QuadKey: {quad_key}")
        else:
            raise ValueError(f"QuadKey not found in dataset: {quad_key}")

    # Merge the GeoJSON files into a single file
    for fn in tmp_fns:
        gdf = gpd.read_file(fn)  # Read each file into a GeoDataFrame
        gdf = gdf[gdf.geometry.within(aoi_shape)]  # Filter geometries within the AOI
        gdf['id'] = range(idx, idx + len(gdf))  # Update 'id' based on idx
        idx += len(gdf)
        combined_gdf = pd.concat([combined_gdf,gdf],ignore_index=True)

100%|██████████| 1/1 [03:12<00:00, 192.99s/it]


## Step 4 - Save the resulting footprints to file

In [13]:
combined_gdf = combined_gdf.to_crs('EPSG:4326')
combined_gdf.to_file(output_fn, driver='GeoJSON')

In [15]:
# Convert to WGS84 (EPSG:4326) if not already in that CRS
combined_gdf = combined_gdf.to_crs('EPSG:4326')

# Define the output directory and filename
output_dir = "C:/Users/Rossouw/OneDrive - Urban Studies/Urban Studies - General/2. Proposals/2025/La Vie Facilities/GlobalMLBuildingFootprints-main/GlobalMLBuildingFootprints-main/Outputs" 
output_filename = "output.geojson"    # Or any name you prefer
output_fn = f"{output_dir}/{output_filename}"

# Save the GeoDataFrame to a GeoJSON file
combined_gdf.to_file(output_fn, driver='GeoJSON')

print(f"File saved successfully to: {output_fn}")

File saved successfully to: C:/Users/Rossouw/OneDrive - Urban Studies/Urban Studies - General/2. Proposals/2025/La Vie Facilities/GlobalMLBuildingFootprints-main/GlobalMLBuildingFootprints-main/Outputs/output.geojson
